In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [2]:
# 读取20220831的所有数据
all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)
'''
title_category_name_tags 3648-->30
description              16457-->70
requirement              15081-->70
position                 6981-->40
skills                   3013-->30
'''

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38613/3095887865.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)


'\ntitle_category_name_tags 3648-->30\ndescription              16457-->70\nrequirement              15081-->70\nposition                 6981-->40\nskills                   3013-->30\n'

In [ ]:
# 将raw_cvjd_20220831_spark.csv的数据精简，只取有用的列
# new_columns = ['cv_id', 'jd_id', 'title', 'category_name', 'recruit_number', 'min_month_salary', 'max_month_salary', 'pay_months', 'tags', 'description', 'requirement', 'currentPosition', ]

In [3]:
all_data

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,updatedBy,updatedTime,uploaderId,version,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,67,2021-12-16 17:25:39,67,1,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,JD20210918160449291911,会员管理经理,2021-10-31 23:59:59.0,130101,用户运营,2c9207157b9b9199017bc895b2f23a39,上海晨光文具股份有限公司,1,...,2c9207157bcd68cf017bf309f6cb003c,2021-09-22 21:00:12,2c9207157bcd68cf017bf309f6cb003c,1,7.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运...",20220831
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,JD20210923104433723714,产品市场经理,2021-10-31 23:59:59.0,110101,产品经理,2c9207157bf94d99017c108765c804aa,凯斯纽荷兰（中国）管理有限公司,1,...,121,2021-10-12 09:51:56,121,1,14.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN,20220831
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,2c9207157bcd68cf017becb0f28b0025,2021-11-23 10:16:45,2c9207157bcd68cf017becb0f28b0025,1,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,28,2021-12-15 13:47:16,28,1,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,JD20220822112125679712,品牌负责人,NaN,130402,运营总监,8a69c4df82b081d30182c37f39d00627,浙江温家味食品科技有限公司,1,...,8a69f61e82926af00182c9b341b36129,2022-08-23 17:15:26,8a69f61e82926af00182c9b341b36129,1,11.0,NaN,"['{""endDate"": ""2012-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-06"", ""companyName"": ""江苏领迅食...","['{""name"": null}']",20220831
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,JD20220818193847553940,大区销售总监,NaN,140402,销售总监,8a69c47b82b07fdc0182b0b5e27a6357,上海普适导航科技股份有限公司,5,...,8a69c4b9820f193401821984f2517399,2022-08-25 13:50:02,8a69c4b9820f193401821984f2517399,1,14.0,NaN,"['{""endDate"": ""2008-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""安徽庆宇光...","['{""name"": null}']",20220831
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,JD20220520123312243433,java开发工程师（不限方向）,NaN,100101,Java,2c92071580b3a1010180dcf9c08d56cc,上海泛微软件有限公司,30,...,2c92071581ec1aab0181f0475082756f,2022-08-22 10:59:17,2c92071581ec1aab0181f0475082756f,1,5.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""泰猎人力(...","['{""description"": ""项目描述: 客服云智库是河南电信客服人员内部使用的一套...",20220831
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,JD20220629161617600853,Legal Senior Manager,NaN,150506,法务经理,2c9207157f347800017f8d372b237e20,曼恩商用车辆贸易（中国）有限公司,1,...,2c9207157f40e97a017f71b1c7c11d59,2022-08-31 18:36:40,2c9207157bf92566017c164de933001c,2,16.0,NaN,"

In [12]:
df = pd.read_csv('../data_20220831/sample_20220831_spark.csv')
df

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_35274/394408529.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data_20220831/sample_20220831_spark.csv')


,Unnamed: 0,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,...,industry_cut,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds
0,0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,C972256920524165184,JD20210910134407574550,drop,1,1,市场策划总监,2021-11-30 23:59:59.0,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['舞蹈', '学']","[0.2290763258934021, -0.14894524216651917, 0.0...","[-0.033116504549980164, -0.058041930198669434,...","['[', '""', 'Ai', '""', ',', '""', 'Ar', '""', ','...","[0.37405160069465637, 0.08108291774988174, -0....","[0.18812726438045502, -0.014001657254993916, -...",20220831
1,1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,C972256920532553817,JD20210916103438584949,link,-1,1,解决方案总监/经理,2021-10-31 23:59:59.0,...,"['通信', '技术', '提供商']","[0.18720732629299164, -0.24131253361701965, -0...","[0.768592119216919, -0.16999222338199615, -0.4...",['计算机技术'],"[0.06550661474466324, -0.30920350551605225, -0...","[0.00835432019084692, -0.6533162593841553, -0....","['[', '""', 'it', '规划', '咨询', '""', ',', '""', '云...","[0.32306215167045593, 0.09080911427736282, -0....","[0.5299833416938782, -0.1909421980381012, -0.2...",20220831
2,2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,C972256920545136700,JD20220216104801688663,drop,1,1,医学博士投资人,NaN,...,"['基金', '/', '证券', '/', '期货', '/', '投资']","[0.022260336205363274, -0.03374471887946129, -...","[0.30257120728492737, -0.31254786252975464, -0...","['生物医学', '工程']","[0.01792692393064499, -0.1300506740808487, -0....","[0.005150213837623596, 0.15490269660949707, -0...","['[', '""', 'VC', '""', ',', '""', 'PPT', '""', ',...","[0.3084435760974884, 0.07542910426855087, -0.2...","[0.09389838576316833, 0.04999205097556114, -0....",20220831
3,3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,C972256920557719589,JD20210908152800055103,reject,2,2,高级用户运营专家,2021-12-31 23:59:59.0,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['市场营销'],"[0.6256594657897949, -0.07856443524360657, 0.0...","[0.7786060571670532, -0.8168095946311951, -0.6...","['[', '""', 'Vi', '""', ',', '""', 'Pr', '""', ','...","[0.3983086943626404, 0.11806842684745789, -0.2...","[0.29188045859336853, -0.04354493319988251, -0...",20220831
4,4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,C972256920557719638,JD20211011173942648108,accept,3,3,咨询总监,2021-10-31 23:59:59.0,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['金融学'],"[0.11331989616155624, -0.1490677297115326, -0....","[-0.3050949275493622, -0.4833742380142212, -2....","['[', '""', 'PPT', '""', ',', '""', 'Word', '""', ...","[0.4078637957572937, 0.07882367819547653, -0.2...","[0.1595805436372757, -0.0552152618765831, -0.3...",20220831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83296,83296,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,C1001558237016887296,JD20220823165719130900,link,-1,1,声学工程师,NaN,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['韩国', '永进', '大学']","[-0.016853807494044304, -0.16189433634281158, ...","[0.4639318585395813, 0.4096287488937378, 0.103...","['CAD', ',', 'Proe', ',', 'Ansys', ',', '优化', ...","[0.29724088311195374, -0.06454341858625412, -0...","[-0.07170484960079193, 0.2823019027709961, -0....",20220831
83297,83297,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,C1002462334712680448,JD20220725172633862905,post,-1,2,销售代表,NaN,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['河南', '科技', '大学']","[0.43664801120758057, -0.4357180

In [2]:
# 读入20220821所有数据
#all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_67195/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [4]:
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [5]:
all_data[['work_age', 'workYear']]

,work_age,workYear
0,5-10年,13.0
1,3-5年,7.0
2,10年以上,14.0
3,3-5年,3.0
4,3-5年,5.0
...,...,...
83296,5年以上,11.0
83297,5-10年,14.0
83298,3-5年,5.0
83299,3-5年,16.0


## 理清需要用的特征


In [6]:
# 这些特征在sample pipeline的表里，但是现在无法下载
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']

# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_month_salary', 'max_month_salary', 'pay_months', 'currentSalary', 'desiredSalary']

# 这些特征的过程已经有了，需要进一步拼接
new_columns = ['position_tfidf_pca', 'skills_tfidf_pca', 'jobTracks_tfidf_pca', 'title_category_tags_tfidf_pca', 'description_tfidf_pca', 'requirement_tfidf_pca', 'work_duration_mean', 'equal_words', 'equal_job']

# 将以上的特征合理分散在wide和deep侧
wide_columns = []
deep_columns = []

In [7]:
all_data[['cv_id', 'jd_id']+raw_columns]

,cv_id,jd_id,age,recruit_number,workYear,min_annual_salary,max_annual_salary,pay_months,currentSalary,desiredSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,80.0,150.0,NaN,NaN,NaN
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,28.0,35.0,NaN,NaN,NaN
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,25.0,35.0,NaN,NaN,NaN
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,60.0,500.0,NaN,NaN,NaN
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,NaN,NaN,12.0,保密,25 - 30k · 15薪
...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,NaN,NaN,12.0,"月薪2.98左右,15薪",合理涨幅
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,NaN,NaN,12.0,12万（10000元/月*,10K~13K
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,NaN,NaN,14.0,18k,20-25k
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,NaN,NaN,13.0,90万,90万


## tfidf_pca

In [1]:
import pandas as pd
# 读入20220821所有数据
all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_28916/358587841.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [8]:
sample = all_data

# 需要将这个向量变成一个列表
def tfidf_pca_merge_fun(series):
    return list(series)

tfidf_pca_file = ['description_tfidf_pca.csv', 'position_tfidf_pca.csv', 'requirement_tfidf_pca.csv', 'skills_tfidf_pca.csv', 'title_category_tags_tfidf_pca.csv']
for csv_file in tfidf_pca_file:
    tfidf_pca = pd.read_csv('../data_20220831/'+csv_file).drop(['Unnamed: 0'], axis=1)
    print(csv_file)
    #print(tfidf_pca)
    col_name = csv_file.strip('.csv') # 把skills的s去掉了
    tfidf_pca[col_name] = tfidf_pca.apply(tfidf_pca_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_pca[[col_name]]], axis=1)

tfidf_pca_col_names = [csv_file.strip('.csv') for csv_file in tfidf_pca_file]
sample[['cv_id', 'jd_id']+tfidf_pca_col_names]

description_tfidf_pca.csv
position_tfidf_pca.csv
requirement_tfidf_pca.csv
skills_tfidf_pca.csv
title_category_tags_tfidf_pca.csv


,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0..."
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23..."
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3..."
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32..."
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00..."
...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1..."
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0..."
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169..."
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0..."


## work_duration

In [9]:
work_duration_mean_df = pd.read_csv('../data_20220831/work_duration_mean.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, work_duration_mean_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000
...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667


## equal_words

In [10]:
equal_job_df = pd.read_csv('../data_20220831/equal_job.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, equal_job_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean'] + ['equal_job']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000,3
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000,4
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000,0
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000,3
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000,1
...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000,4
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333,3
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000,5
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667,1


## salary

In [11]:
salary_df = pd.read_csv('../data_20220831/salary.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, salary_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean'] + ['equal_job'] + ['parsed_desiredSalary', 'parsed_currentSalary']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000,3,-1,-1
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000,4,-1,-1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000,0,-1,-1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000,3,-1,-1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000,1,450,-1
...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000,4,-1,-1
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333,3,120,120
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000,5,300,216
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667,1,900,900


In [13]:
sample[columns].to_csv('../data_20220831/sample_new_feature_20220831.csv')

In [16]:
df = pd.read_csv('../data_20220831/sample_new_feature_20220831.csv').drop('Unnamed: 0', axis=1)

In [17]:
df

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000,3,-1,-1
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000,4,-1,-1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000,0,-1,-1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000,3,-1,-1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000,1,450,-1
...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000,4,-1,-1
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333,3,120,120
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000,5,300,216
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667,1,900,900


In [ ]:
'''
特别注意，以上的拼接都是基于raw_cvjd_20220831_spark.csv数据

description_tfidf_pca
position_tfidf_pca
requirement_tfidf_pca
kills_tfidf_pca
title_category_tags_tfidf_pca
work_duration_mean
equal_job
parsed_desiredSalary
parsed_currentSalary
这些列没有cv_id和jd_id，仅仅是通过行拼接生成数据集（应该是没有行数据错位的情况）。
'''

## 原始特征

In [24]:
# 这些特征在sample pipeline的表里，但是现在无法下载
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']

# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_month_salary', 'max_month_salary', 'pay_months']

# 这些特征的过程已经有了，需要进一步拼接
new_columns = ['position_tfidf_pca', 'skills_tfidf_pca', 'jobTracks_tfidf_pca', 'title_category_tags_tfidf_pca', 'description_tfidf_pca', 'requirement_tfidf_pca', 'work_duration_mean', 'equal_job', 'parsed_currentSalary', 'parsed_desiredSalary']

# 将以上的特征合理分散在wide和deep侧
wide_columns = []
deep_columns = []

In [27]:
sample[['cv_id', 'jd_id'] + raw_columns].to_csv('../data_20220831/sample_raw_feature_20220831.csv')

In [28]:
pd.read_csv('../data_20220831/sample_raw_feature_20220831.csv')

,Unnamed: 0,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months
0,0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,NaN,NaN,NaN
1,1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,NaN,NaN,NaN
2,2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,NaN,NaN,NaN
3,3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,NaN,NaN,NaN
4,4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0
...,...,...,...,...,...,...,...,...,...
83296,83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0
83297,83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0
83298,83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0
83299,83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0


## ext特征

In [ ]:
# 这些特征在sample pipeline的表里，但是现在无法下载
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']

# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_month_salary', 'max_month_salary', 'pay_months']

# 这些特征的过程已经有了，需要进一步拼接
new_columns = ['position_tfidf_pca', 'skills_tfidf_pca', 'jobTracks_tfidf_pca', 'title_category_tags_tfidf_pca', 'description_tfidf_pca', 'requirement_tfidf_pca', 'work_duration_mean', 'equal_job', 'parsed_currentSalary', 'parsed_desiredSalary']

# 将以上的特征合理分散在wide和deep侧
wide_columns = []
deep_columns = []

In [29]:
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38613/893757878.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1)


In [33]:
all_sample[['cv_id', 'jd_id'] + ext_columns].to_csv('../data_20220831/sample_ext_feature_20220831.csv')

In [35]:
pd.read_csv('../data_20220831/sample_ext_feature_20220831.csv')

,Unnamed: 0,cv_id,jd_id,school_labels_cv,school_labels_jd,expectlocation_range_cv,location_range_jd,degree_index_cv,degree_index_jd,workyear_range_jd,gender_index_cv,job_intentstatus_cv
0,0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,[],[],['未知市'],['深圳市'],5,5,"[5, 10]",0,3
1,1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,[],[],['北京市'],['长沙市'],6,5,"[10, 100]",0,3
2,2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,"['985', '211', '双一流', 'qs100']",[],['北京市'],[],6,7,"[0, 1]",0,3
3,3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,[],[],['未知市'],['杭州市'],5,5,"[5, 10]",1,3
4,4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,"['985', '211', '双一流']",[],['未知市'],['深圳市'],5,5,"[5, 10]",0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
83296,83296,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,[],[],['深圳市'],[],5,3,"[3, 5]",0,3
83297,83297,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,[],[],['上海市'],[],5,3,"[1, 3]",0,2
83298,83298,8a69d7c282199453018247bb00cb68a3,8a69c468826ee9b40182831bedfc5e79,"['985', '211', '双一流', 'qs100']",[],['上海市'],[],6,5,"[10, 100]",0,2
83299,83299,8a69d7c282199453018247bb00cb68a3,8a69f6f8823a93b9018247c30aca4e3d,"['985', '211', '双一流', 'qs100']",['不限'],['上海市'],[],6,5,"[10, 100]",0,2


## 拼接数据集

In [124]:
# 初步怀疑这里的feature的cvid和jdid有重复
base_path = '../data_20220831/'
raw_feature_file = 'sample_raw_feature_20220831.csv'
ext_feature_file = 'sample_ext_feature_20220831.csv'
new_feature_file = 'sample_new_feature_20220831.csv'

# 去重
raw_feature = pd.read_csv(base_path + raw_feature_file).drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')
ext_feature = pd.read_csv(base_path + ext_feature_file).drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')
new_feature = pd.read_csv(base_path + new_feature_file).drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')

In [125]:
raw_feature.sort_values(by=['cv_id', 'jd_id'])

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months
30231,2c9207157b9492d2017b9f4220f6013a,2c9207157b9b9199017b9f34d103029b,28.0,1,5.0,NaN,NaN,NaN
30626,2c9207157b9492d2017b9f465ae1013e,2c9207157ef7ed16017f0146624d4d9e,35.0,1,13.0,25.0,45.0,14.0
24113,2c9207157b9492d2017b9f4e9f240151,2c9207157f93e289017f974c19df2381,32.0,1,10.0,100.0,125.0,12.0
8979,2c9207157b9492d2017b9f6f46a40181,2c9207157b9b9199017b9f34d103029b,31.0,1,8.0,NaN,NaN,NaN
14779,2c9207157b9492d2017b9ff4441b018f,2c9207157bcbb76b017bce3d99ba059a,41.0,1,5.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
33565,8a69f68b828a2cd9018291b4742b5196,8a69f6f8823a93b901823da3a4c80884,34.0,1,13.0,40.0,70.0,13.0
65393,8a69f68b828a2cd9018291b81ab1557d,8a69c4a8820f0ee00182143cd83b2916,31.0,1,8.0,15.0,35.0,17.0
46743,8a69f68b828a2cd9018291baf9695859,2c920715816798cb018170a5990e043e,40.0,9,16.0,10.0,30.0,17.0
37701,8a69f68b828a2cd9018291bce8935a65,8a69f68e82198e9101821a3265a21e9c,35.0,4,13.0,NaN,NaN,NaN


In [126]:
ext_feature.sort_values(by=['cv_id', 'jd_id'])

,cv_id,jd_id,school_labels_cv,school_labels_jd,expectlocation_range_cv,location_range_jd,degree_index_cv,degree_index_jd,workyear_range_jd,gender_index_cv,job_intentstatus_cv
54238,2c9207157b9492d2017b9f4220f6013a,2c9207157b9b9199017b9f34d103029b,"['985', '211', '双一流']",[],['厦门市'],['厦门市'],5,5,"[3, 5]",0,3
30596,2c9207157b9492d2017b9f465ae1013e,2c9207157ef7ed16017f0146624d4d9e,[],['双一流'],['厦门市'],[],5,5,"[5, 10]",0,3
77230,2c9207157b9492d2017b9f4e9f240151,2c9207157f93e289017f974c19df2381,"['985', '211', '双一流']",[],['厦门市'],[],5,5,"[10, 100]",0,3
73447,2c9207157b9492d2017b9f6f46a40181,2c9207157b9b9199017b9f34d103029b,[],[],['未知市'],['厦门市'],5,5,"[3, 5]",1,3
23684,2c9207157b9492d2017b9ff4441b018f,2c9207157bcbb76b017bce3d99ba059a,"['985', '211', '双一流', 'qs100']",[],['未知市'],['深圳市'],6,5,"[5, 10]",1,3
...,...,...,...,...,...,...,...,...,...,...,...
20113,8a69f68b828a2cd9018291b4742b5196,8a69f6f8823a93b901823da3a4c80884,[],[],['广州市'],[],5,5,"[5, 10]",0,0
82951,8a69f68b828a2cd9018291b81ab1557d,8a69c4a8820f0ee00182143cd83b2916,[],[],['深圳市'],[],5,5,"[5, 10]",0,2
51396,8a69f68b828a2cd9018291baf9695859,2c920715816798cb018170a5990e043e,"['985', '211', '双一流']",[],['上海市'],[],5,3,"[5, 10]",0,2
39021,8a69f68b828a2cd9018291bce8935a65,8a69f68e82198e9101821a3265a21e9c,[],[],['上海市'],['上海市'],5,5,"[5, 10]",0,2


In [127]:
new_feature.sort_values(by=['cv_id', 'jd_id'])

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
30231,2c9207157b9492d2017b9f4220f6013a,2c9207157b9b9199017b9f34d103029b,"[0.0599345361662065, -0.0767967095025605, -0.0...","[-0.1054037789047835, -0.0104185279968552, -0....","[-0.0470596219162773, 0.1130961797826841, -0.0...","[-0.0950710757884233, -0.1473097543095731, -0....","[0.0676997430068378, -0.0492043457317328, -0.0...",608.00,1,96,75
30626,2c9207157b9492d2017b9f465ae1013e,2c9207157ef7ed16017f0146624d4d9e,"[0.026291432620492, -0.0368446460107445, 0.043...","[0.1536565004556754, 0.0014335952630696, -0.08...","[-0.0217293325987802, -0.131824125392145, -0.0...","[0.1692623701366227, -0.1512692056289587, 0.28...","[0.0575216915362041, -0.0506365443053506, -0.0...",1004.00,1,288,-1
24113,2c9207157b9492d2017b9f4e9f240151,2c9207157f93e289017f974c19df2381,"[-0.0005118874794016, -0.0561343829275314, 0.1...","[0.1009024099895537, 0.2587870453893555, -0.11...","[-0.0868734992350594, 0.0746479032403449, 0.03...","[-0.0433432666647966, -0.0197585374234068, -0....","[0.0220240902971522, -0.0780113564594292, -0.0...",2345.00,3,324,-1
8979,2c9207157b9492d2017b9f6f46a40181,2c9207157b9b9199017b9f34d103029b,"[0.0599345361662065, -0.0767967095025605, -0.0...","[-0.1466987216702775, 0.040266508540216, -0.09...","[-0.0470596219162773, 0.1130961797826841, -0.0...","[-0.0950710757884233, -0.1473097543095731, -0....","[0.0676997430068378, -0.0492043457317328, -0.0...",516.50,2,120,93
14779,2c9207157b9492d2017b9ff4441b018f,2c9207157bcbb76b017bce3d99ba059a,"[-0.1251006281289015, 0.059732753775596, -0.00...","[-0.0371431976655152, -0.0879360569284861, -0....","[-0.0635984771163071, 0.1566419781432294, -0.0...","[0.0554130200076727, -0.0057938254195516, -0.1...","[-0.0310672875308255, -0.0891909188893691, 0.0...",NaN,2,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
33565,8a69f68b828a2cd9018291b4742b5196,8a69f6f8823a93b901823da3a4c80884,"[-0.1411348726635974, 0.1545761397914054, 0.01...","[0.0459260775467377, -0.2299206580557053, 0.51...","[-0.1104159399085872, -0.055311944519666, -0.0...","[0.2184741887163782, -0.0527406742271523, 0.00...","[-0.2081075561923475, -0.0360057063377714, 0.2...",1461.00,4,1200,2000
65393,8a69f68b828a2cd9018291b81ab1557d,8a69c4a8820f0ee00182143cd83b2916,"[0.02232992517464, -0.0961146616323217, -0.042...","[0.0182234065077543, -0.079240344788645, -0.07...","[-0.0536113201353703, 0.0460421057886939, -0.1...","[-0.0021967363577278, -0.1085157190128784, 0.0...","[0.0002822125983735, -0.0531525328662003, -0.0...",1278.50,4,700,340
46743,8a69f68b828a2cd9018291baf9695859,2c920715816798cb018170a5990e043e,"[-0.1521925580870764, -0.0387579263375424, 0.0...","[-0.0313648785410539, -0.0833861124689416, -0....","[-0.1054951893602227, -0.0377046978851067, 0.1...","[0.3668429235412169, 0.1468171072898762, 0.092...","[-0.0028890579937088, -0.1362721144266969, 0.0...",726.75,3,327,252
37701,8a69f68b828a2cd9018291bce8935a65,8a69f68e82198e9101821a3265a21e9c,"[-0.0791245377151637, -0.202603700067891, -0.2...","[0.0693369750991358, -0.111781639912603, -0.08...","[-0.1033812561532749, 0.4019940419188615, -0.1...","[0.187490031298046, -0.0990302460478618, 0.291...","[-0.0123565703908655, -0.1743496739999127, -0....",1217.40,3,192,180


In [128]:
medium = raw_feature.join(ext_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature = medium.join(new_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,job_intentstatus_cv,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,NaN,NaN,NaN,[],[],...,3,"[-0.2028268203396756, 0.052902330097395, 0.064...","[-0.0393549614722772, -0.1145062990375059, -0....","[-0.0885833919216516, 0.1251912920110739, -0.0...","[0.332299671725948, 0.1661089355047844, 0.1869...","[-0.0428931610734227, -0.1624195901085068, 0.0...",1209.500000,3,-1,-1
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,NaN,NaN,NaN,[],[],...,2,"[-0.0930831222487631, 0.0519592769164767, -0.1...","[0.1053468947433308, -0.0266945757513419, -0.0...","[-0.0801264640475711, 0.122717540913069, -0.01...","[0.2361554447739323, 0.1126566470966733, -0.24...","[-0.2347225490786922, 0.0825676460870139, 0.23...",506.000000,4,-1,-1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,NaN,NaN,NaN,[],[],...,3,"[-0.077256404167103, -0.195452028548207, -0.22...","[-0.0368041780176452, -0.0857596688737137, -0....","[-0.0644508985020475, 0.1611090469848189, -0.0...","[-0.0922030791609062, -0.1406030646715831, -0....","[-0.1959575316104875, 0.2008839134718935, -0.3...",811.500000,0,-1,-1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,NaN,NaN,NaN,"['211', '双一流']",[],...,3,"[-0.0933581483833488, 0.0982068787794115, -0.0...","[0.0490471624206398, -0.0409758387316529, -0.0...","[-0.1339726610662708, -0.1761763831994133, 0.0...","[0.1429396280466366, 0.0217622310718796, -0.14...","[-0.3678224323290787, 0.1854520957699004, 0.32...",76.500000,3,-1,-1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,3,"[0.0391819610653473, -0.0287343543993758, 0.06...","[-0.0276426492329401, -0.111924304908804, -0.0...","[-0.0136365894358371, -0.0993886855789762, -0....","[-0.1003025481350853, -0.1495209206927971, -0....","[0.0062176220898476, -0.055003735259901, -0.00...",1034.000000,1,450,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,0,"[-0.1614955741689255, 0.0982571120950793, 0.02...","[-0.0025833865349725, -0.2508193885760124, 0.0...","[-0.1517178304086961, -0.1846200489563693, 0.1...","[0.3241805444120278, -0.0345988526341208, 0.26...","[-0.0954686272882023, -0.0493724436404325, 0.1...",1268.000000,4,-1,-1
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,3,"[-0.187368032369494, 0.0815596725923325, 0.018...","[0.1067952861660345, -0.225099081085894, -0.06...","[-0.1015761687785662, 0.0302194323476557, 0.04...","[0.0106236588110783, -0.1609745502874504, -0.0...","[-0.1288286036514234, -0.3332390781216969, 0.0...",1583.333333,3,120,120
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,0,"[0.4185259318945861, -0.0453746812279699, 0.39...","[-0.4597675655720933, 0.5738247473834591, 0.33...","[0.5440054441064249, -0.080866345740466, 0.135...","[-0.2612896440159119, 0.0846500996922437, 0.11...","[0.5515111960481315, 0.4001609626530521, 0.169...",487.000000,5,300,216
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,3,"[-0.0823327926841566, -0.2134441983861008, -0....","[-0.0126004164756376, -0.1818836719727035, -0....","[-0.0904644960948339, 0.3290683312687502, -0.0...","[0.079180668535461, -0.141120307479467, 0.0457...","[0.0103941678651557, -0.0621128773282237, -0.0...",968.666667,1,900,900


In [129]:
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')
all_sample

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38613/2448397297.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first')


,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,category_id,...,industry_cut,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,C972256920524165184,JD20210910134407574550,drop,1,1,市场策划总监,2021-11-30 23:59:59.0,140404,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['舞蹈', '学']","[0.2290763258934021, -0.14894524216651917, 0.0...","[-0.033116504549980164, -0.058041930198669434,...","['[', '""', 'Ai', '""', ',', '""', 'Ar', '""', ','...","[0.37405160069465637, 0.08108291774988174, -0....","[0.18812726438045502, -0.014001657254993916, -...",20220831
1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,C972256920532553817,JD20210916103438584949,link,-1,1,解决方案总监/经理,2021-10-31 23:59:59.0,101201,...,"['通信', '技术', '提供商']","[0.18720732629299164, -0.24131253361701965, -0...","[0.768592119216919, -0.16999222338199615, -0.4...",['计算机技术'],"[0.06550661474466324, -0.30920350551605225, -0...","[0.00835432019084692, -0.6533162593841553, -0....","['[', '""', 'it', '规划', '咨询', '""', ',', '""', '云...","[0.32306215167045593, 0.09080911427736282, -0....","[0.5299833416938782, -0.1909421980381012, -0.2...",20220831
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,C972256920545136700,JD20220216104801688663,drop,1,1,医学博士投资人,NaN,180101,...,"['基金', '/', '证券', '/', '期货', '/', '投资']","[0.022260336205363274, -0.03374471887946129, -...","[0.30257120728492737, -0.31254786252975464, -0...","['生物医学', '工程']","[0.01792692393064499, -0.1300506740808487, -0....","[0.005150213837623596, 0.15490269660949707, -0...","['[', '""', 'VC', '""', ',', '""', 'PPT', '""', ',...","[0.3084435760974884, 0.07542910426855087, -0.2...","[0.09389838576316833, 0.04999205097556114, -0....",20220831
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,C972256920557719589,JD20210908152800055103,reject,2,2,高级用户运营专家,2021-12-31 23:59:59.0,130405,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['市场营销'],"[0.6256594657897949, -0.07856443524360657, 0.0...","[0.7786060571670532, -0.8168095946311951, -0.6...","['[', '""', 'Vi', '""', ',', '""', 'Pr', '""', ','...","[0.3983086943626404, 0.11806842684745789, -0.2...","[0.29188045859336853, -0.04354493319988251, -0...",20220831
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,C972256920557719638,JD20211011173942648108,accept,3,3,咨询总监,2021-10-31 23:59:59.0,260401,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['金融学'],"[0.11331989616155624, -0.1490677297115326, -0....","[-0.3050949275493622, -0.4833742380142212, -2....","['[', '""', 'PPT', '""', ',', '""', 'Word', '""', ...","[0.4078637957572937, 0.07882367819547653, -0.2...","[0.1595805436372757, -0.0552152618765831, -0.3...",20220831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,C1001558237016887296,JD20220823165719130900,link,-1,1,声学工程师,NaN,100801,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['韩国', '永进', '大学']","[-0.016853807494044304, -0.16189433634281158, ...","[0.4639318585395813, 0.4096287488937378, 0.103...","['CAD', ',', 'Proe', ',', 'Ansys', ',', '优化', ...","[0.29724088311195374, -0.06454341858625412, -0...","[-0.07170484960079193, 0.2823019027709961, -0....",20220831
83297,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,C1002462334712680448,JD20220725172633862905,post,-1,2,销售代表,NaN,140301,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['河南', '科技', '大学']","[0.4

In [132]:
label_df = all_sample[['cv_id', 'jd_id', 'da_label', 'rank_label']]
raw_ext_new_feature_label = raw_ext_new_feature.join(label_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature_label.columns
raw_ext_new_feature_label.to_csv('../data_20220831/raw_ext_new_feature_label.csv')

## 复推数据标签

In [ ]:
# 找到哪一份数据集有复推标签

#audit_state（4未处理：过滤，5放弃不会有order，6同意与101对应）
#resume_type: 2-AI人岗, 5-资深人工撮合, 6-复推, 普通猎头单不在其中（普通猎头正样本要去order里找）
#compatibility 规则打分(0-100)，AI人岗类型(resume_type=2)有效
#resume_source：忽略，resume_type=2，5，6时全都等于2，代表AI人岗和复推



